## Ingest the list of contributors

In [11]:
import uuid
import pandas as pd
import sqlalchemy as salc
import json

In [6]:
df = pd.read_csv('contrib_pagerank_scores.csv')

In [7]:
df

,contrib_id,normalized_pagerank_score
0,010002df-6100-0000-0000-000000000000,20.000000
1,010000fd-e400-0000-0000-000000000000,19.052353
2,01002a5e-b400-0000-0000-000000000000,17.138503
3,0100034c-5400-0000-0000-000000000000,16.901099
4,01003291-1b00-0000-0000-000000000000,16.831682
...,...,...
1431,0100b291-e700-0000-0000-000000000000,5.000065
1432,0100b784-6000-0000-0000-000000000000,5.000064
1433,010043be-3100-0000-0000-000000000000,5.000000
1434,0105ff97-1500-0000-0000-000000000000,5.000000


In [8]:
cntrb_set = list(df['contrib_id'])

In [12]:
with open("wasm_creds.json") as config_file:
    config = json.load(config_file)

In [13]:
database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [14]:
prr_contrib = pd.DataFrame()

for cntrb_id in cntrb_set:
    repo_query = salc.sql.text(f"""
                SET SCHEMA 'augur_data';
                SELECT *
                FROM
                contributor_repo c
        """)

with engine.connect() as conn:
    df_current_repo = pd.read_sql_query(repo_query, conn)

In [15]:
df_current_repo.head()

,cntrb_repo_id,repo_git,repo_name,gh_repo_id,cntrb_category,event_id,created_at,tool_source,tool_version,data_source,data_collection_date,cntrb_id
0,34198358,https://api.github.com/repos/IAMSUPERMONKEY/fl...,IAMSUPERMONKEY/flutter_easyrefresh_v2_flutter3,500325907,WatchEvent,31180979155,2023-08-17 10:35:15,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-17 06:11:02,01009668-df00-0000-0000-000000000000
1,51320997,https://api.github.com/repos/ramosisw/dotnet-sort,ramosisw/dotnet-sort,177522926,CreateEvent,31839353755,2023-09-14 15:18:51,Contributor Breadth Worker,0.0.1,GitHub API,2023-09-27 12:13:02,01003372-5100-0000-0000-000000000000
2,34198359,https://api.github.com/repos/Fitem/flutter_uti...,Fitem/flutter_util_code,647552683,WatchEvent,31180261079,2023-08-17 10:07:09,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-17 06:11:02,01009668-df00-0000-0000-000000000000
3,23320398,https://api.github.com/repos/genabdulrehman/cr...,genabdulrehman/credit-card-animation,562882825,WatchEvent,31141459050,2023-08-16 00:35:46,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-15 21:08:30,01009668-df00-0000-0000-000000000000
4,23320399,https://api.github.com/repos/smiling1990/Flutt...,smiling1990/FlutterCards,228790938,WatchEvent,31141439717,2023-08-16 00:34:10,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-15 21:08:30,01009668-df00-0000-0000-000000000000


In [16]:
def convert_to_uuid(string_list):
    uuid_list = [uuid.UUID(string) for string in string_list]
    return uuid_list

uuid_list = convert_to_uuid(cntrb_set)

In [17]:
len(cntrb_set)

1436

In [18]:
len(list(set(uuid_list).intersection(set(list(df_current_repo['cntrb_id'])))))

1269

From the total number of contributor IDs that we received, above are present in this augur database

In [ ]:
# TODO: inspect why query below does not work
# where we automatically check contributor IDs that are present in the DB and select details only for those
# instead of getting it for all contributors

# prr_contrib = pd.DataFrame()

# for cntrb_id in uuid_list:
#     repo_query = salc.sql.text(f"""
#                 SET SCHEMA 'augur_data';
#                 SELECT *
#                 FROM
#                 contributor_repo c
#                 WHERE
#                 c.cntrb_id = \'{cntrb_id}\'
#         """)

# with engine.connect() as conn:
#     df_repo = pd.read_sql_query(repo_query, conn)

In [ ]:
df_current_repo

,cntrb_repo_id,repo_git,repo_name,gh_repo_id,cntrb_category,event_id,created_at,tool_source,tool_version,data_source,data_collection_date,cntrb_id
0,34198358,https://api.github.com/repos/IAMSUPERMONKEY/fl...,IAMSUPERMONKEY/flutter_easyrefresh_v2_flutter3,500325907,WatchEvent,31180979155,2023-08-17 10:35:15,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-17 06:11:02,01009668-df00-0000-0000-000000000000
1,34198359,https://api.github.com/repos/Fitem/flutter_uti...,Fitem/flutter_util_code,647552683,WatchEvent,31180261079,2023-08-17 10:07:09,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-17 06:11:02,01009668-df00-0000-0000-000000000000
2,23320398,https://api.github.com/repos/genabdulrehman/cr...,genabdulrehman/credit-card-animation,562882825,WatchEvent,31141459050,2023-08-16 00:35:46,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-15 21:08:30,01009668-df00-0000-0000-000000000000
3,23320399,https://api.github.com/repos/smiling1990/Flutt...,smiling1990/FlutterCards,228790938,WatchEvent,31141439717,2023-08-16 00:34:10,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-15 21:08:30,01009668-df00-0000-0000-000000000000
4,1331068,https://api.github.com/repos/mydeveloperday/sp...,mydeveloperday/spectral-sarif,643239814,PushEvent,29206154815,2023-05-21 12:46:42,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-08 01:07:25,0100f4d9-0a00-0000-0000-000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2675309,47850154,https://api.github.com/repos/pydantic/pydantic,pydantic/pydantic,90194616,IssueCommentEvent,31227754656,2023-08-19 09:29:08,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-23 15:23:30,010001b0-ad00-0000-0000-000000000000
2675310,47850155,https://api.github.com/repos/projg2/eclean-kernel,projg2/eclean-kernel,113657504,IssueCommentEvent,31226690465,2023-08-19 06:26:43,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-23 15:23:30,010001b0-ad00-0000-0000-000000000000
2675311,8324154,https://api.github.com/repos/mgorny/cattrs,mgorny/cattrs,677779896,DeleteEvent,31085311697,2023-08-13 15:15:54,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-13 14:11:08,010001b0-ad00-0000-0000-000000000000
2675312,2684623,https://api.github.com/repos/mgorny/httpbin,mgorny/httpbin,673318958,PushEvent,31080632995,2023-08-13 02:49:10,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-12 22:14:30,010001b0-ad00-0000-0000-000000000000


In [ ]:
filtered_df = df_current_repo[df_current_repo['cntrb_id'].isin(uuid_list)]

In [ ]:
filtered_df

,cntrb_repo_id,repo_git,repo_name,gh_repo_id,cntrb_category,event_id,created_at,tool_source,tool_version,data_source,data_collection_date,cntrb_id
318,21341265,https://api.github.com/repos/microsoft/onnxrun...,microsoft/onnxruntime,156939672,IssueCommentEvent,31136723492,2023-08-15 19:38:24,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-15 15:15:09,01023116-4900-0000-0000-000000000000
319,21341266,https://api.github.com/repos/microsoft/onnxrun...,microsoft/onnxruntime,156939672,IssueCommentEvent,31136696038,2023-08-15 19:37:01,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-15 15:15:09,01023116-4900-0000-0000-000000000000
320,21341267,https://api.github.com/repos/microsoft/onnxrun...,microsoft/onnxruntime,156939672,IssueCommentEvent,31136659750,2023-08-15 19:35:11,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-15 15:15:09,01023116-4900-0000-0000-000000000000
322,39081997,https://api.github.com/repos/martindevans/LLam...,martindevans/LLamaSharp,668766844,CreateEvent,31197789938,2023-08-17 22:26:26,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-17 20:08:59,010002b5-6f00-0000-0000-000000000000
323,37305053,https://api.github.com/repos/martindevans/LLam...,martindevans/LLamaSharp,668766844,PushEvent,31193438028,2023-08-17 18:35:17,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-17 15:12:35,010002b5-6f00-0000-0000-000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2674803,408798,https://api.github.com/repos/nomic-ai/gpt4all,nomic-ai/gpt4all,619959033,IssueCommentEvent,29687374882,2023-06-12 11:17:41,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-07 00:10:36,0104abd9-4c00-0000-0000-000000000000
2674804,408824,https://api.github.com/repos/nomic-ai/gpt4all,nomic-ai/gpt4all,619959033,IssueCommentEvent,29551549794,2023-06-06 05:03:34,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-07 00:10:36,0104abd9-4c00-0000-0000-000000000000
2674805,408825,https://api.github.com/repos/nomic-ai/gpt4all,nomic-ai/gpt4all,619959033,IssueCommentEvent,29522269929,2023-06-05 03:46:52,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-07 00:10:36,0104abd9-4c00-0000-0000-000000000000
2675096,411166,https://api.github.com/repos/webview/webview,webview/webview,100781035,IssuesEvent,30289104367,2023-07-09 03:30:58,Contributor Breadth Worker,0.0.1,GitHub API,2023-08-07 00:11:53,01003537-4200-0000-0000-000000000000


In [ ]:
set(filtered_df['repo_git'])

{'https://api.github.com/repos/fugue-project/triad',
 'https://api.github.com/repos/GuillaumeGomez/sysinfo',
 'https://api.github.com/repos/go-skynet/go-llama.cpp',
 'https://api.github.com/repos/diowa/ruby2-rails5-bootstrap-heroku',
 'https://api.github.com/repos/zebedeeio/wavlake-zbd-social-embed',
 'https://api.github.com/repos/inkeliz/go_inkwasm',
 'https://api.github.com/repos/anchan828/nest-bull',
 'https://api.github.com/repos/itsthomthomob/Spring-2024-New-Grads',
 'https://api.github.com/repos/trustedshops-public/mail-blocklist-monitor',
 'https://api.github.com/repos/JoneQin/AutoGreen',
 'https://api.github.com/repos/adithyaakrishna/rh-ts-bot',
 'https://api.github.com/repos/django/channels',
 'https://api.github.com/repos/wxx9248/baijiayunxiao-downloader',
 'https://api.github.com/repos/elyshaffir/wgsl_preprocessor',
 'https://api.github.com/repos/crablang/crab',
 'https://api.github.com/repos/it985/auto-green',
 'https://api.github.com/repos/hyunbin-and-ddakkarri/food-finder

In [ ]:
len(set(filtered_df['repo_git']))

38269

The core contributors have contributed to above number of repositories, out of the above repos, filter out repos based on various criteria. Keep repos which are worth exploring and include them in Augur.